In [29]:
import glob
import fitz
import os
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
import numpy as np
import cv2
import shutil

In [30]:
def collect_filename(dir):
    file = glob.glob(dir + '*.jpg')
    filename = []
    for i in file:
        filename.append(i.replace('\\', '/'))
    return filename

In [31]:
def cut_title(file):##cut the useless title in the picture, the length of huawei p20 is 170
    img = cv2.imread(file)
    size = img.shape
    cropped = img[int(size[1]/810*170):size[0], 0:size[1]]
    if os.path.exists('picture/temporary/') == False:
        os.mkdir('picture/temporary/')
    cv2.imwrite('picture/temporary/' + 'a.jpg', cropped)

In [32]:
def split_picture(filename = 'picture/temporary/a.jpg', storage_folder = 'picture/temporary/'):#the length in huawei p20 is 470
    img = cv2.imread(filename)
    size = img.shape
    page = 1
    ppt_length = int(size[1]/810*470)
    while True:
        cropped = img[ppt_length*(page-1):ppt_length*page, 0:size[1]]
        cv2.imwrite(storage_folder + str(page) + '.jpg', cropped)
        if ppt_length*page>size[0]:
            break
        page=page+1

In [33]:
def pic_to_pdf(pdfname, picture_folder = 'picture/temporary/', storage_folder = 'pdf/'):
    if os.path.exists(storage_folder + 'temporary/') == False:
        os.mkdir(storage_folder + 'temporary/')
    file = glob.glob(picture_folder+'*.jpg')
    filename = []
    for i in file:
        filename.append(i.replace('\\', '/'))
    filename.remove(picture_folder + 'a.jpg')
    filename = sorted(filename, key= lambda x: int(x[len(picture_folder):-4]))
    sort = 1
    for img in filename:
        doc = fitz.open()
        imgdoc = fitz.open(img)
        pdf = imgdoc.convert_to_pdf()
        doc.insert_pdf(fitz.open('pdf', pdf))
        doc.save(storage_folder+ 'temporary/' + str(sort) + '.pdf')
        sort = sort + 1
        doc.close()
    pdf_file = glob.glob(storage_folder + 'temporary/' + '*.pdf')
    pdf_filename = []
    for i in pdf_file:
        pdf_filename.append(i.replace('\\', '/'))
    pdf_filename = sorted(pdf_filename, key= lambda x: int(x[len(storage_folder+ 'temporary/'):-4]))
    pdf_merger = PdfFileMerger()
    for pdf in pdf_filename:
        f = open(pdf, 'rb')
        pdf_merger.append(f)
        f.close()
    pdf_merger.write(storage_folder + pdfname + '.pdf')

In [34]:
def remove_temporaryfile(trash_filename = ['picture/temporary/', 'pdf/temporary/']):
    # picture_temporary_file = glob.glob(trash_filename[0]+'*.jpg')
    # pdf_temporary_file = glob.glob(trash_filename[1])
    shutil.rmtree(trash_filename[0])
    shutil.rmtree(trash_filename[1])

In [36]:
dir = 'picture/'
for i in collect_filename(dir):
    cut_title(i)
    split_picture()
    pic_to_pdf(i[len(dir):-4])
    remove_temporaryfile()